<a href="https://colab.research.google.com/github/kusuma-103/Dynamic-parking-pricing/blob/main/notebook/pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pathway

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 

In [ ]:
import numpy as np
import pandas as pd
import pathway as pw
from bokeh.plotting import figure, show, output_notebook
from bokeh.io import push_notebook
from bokeh.models import ColumnDataSource
from math import radians, cos, sin, asin, sqrt

output_notebook()


In [ ]:
# Simulated Data (structure according to PDF)
np.random.seed(42)
n_parking_lots = 14
n_time_steps_per_lot = 18 * 73  # 18 points per day × 73 days for each parking lot

# Create all combinations of ParkingLotID and TimeStep
parking_lot_ids = range(1, n_parking_lots + 1)
time_steps = range(n_time_steps_per_lot)

# Use itertools.product to get all combinations
from itertools import product
combinations = list(product(parking_lot_ids, time_steps))

# Total number of data points
total_data_points = len(combinations)

# Create the DataFrame using the combinations
data = pd.DataFrame(combinations, columns=['ParkingLotID', 'TimeStep'])

# Generate other columns with the same length as the total data points
data['Latitude'] = np.random.uniform(12.9, 13.1, total_data_points)
data['Longitude'] = np.random.uniform(77.5, 77.7, total_data_points)
data['Capacity'] = np.random.randint(30, 100, total_data_points)
data['Occupancy'] = np.random.randint(0, 100, total_data_points)
data['QueueLength'] = np.random.randint(0, 10, total_data_points)
data['VehicleType'] = np.random.choice(['car', 'bike', 'truck'], total_data_points)
data['TrafficLevel'] = np.random.uniform(0, 1, total_data_points)
data['IsSpecialDay'] = np.random.choice([0, 1], total_data_points)

# Display the first few rows of the generated data
display(data.head())

,ParkingLotID,TimeStep,Latitude,Longitude,Capacity,Occupancy,QueueLength,VehicleType,TrafficLevel,IsSpecialDay
0,1,0,12.974908,77.575090,64,2,4,truck,0.719632,1
1,1,1,13.090143,77.570635,69,68,1,truck,0.644363,0
2,1,2,13.046399,77.665395,31,13,2,bike,0.033390,0
3,1,3,13.019732,77.501912,32,7,9,truck,0.778205,1
4,1,4,12.931204,77.659589,43,88,3,truck,0.207193,0


In [ ]:
def baseline_linear_model(price, occupancy, capacity, alpha=0.1):
    occupancy_rate = occupancy / capacity
    new_price = price + alpha * occupancy_rate
    return new_price

# Example usage of the function:
calculated_price = baseline_linear_model(price=10.0, occupancy=50, capacity=100)
print(calculated_price)

10.05


In [ ]:
def demand_based_model(base_price, occupancy, capacity, queue, traffic, special_day, vehicle_type):
    vehicle_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
    vehicle_weight = vehicle_weights.get(vehicle_type, 1.0)

    demand = (0.5 * (occupancy / capacity) +
              0.3 * queue -
              0.2 * traffic +
              0.1 * special_day +
              0.1 * vehicle_weight)

    normalized_demand = np.clip((demand - 0.5) / 2, -0.5, 1)
    price = base_price * (1 + 0.5 * normalized_demand)
    price = np.clip(price, base_price * 0.5, base_price * 2)
    return price


In [ ]:
# Example usage of the demand_based_model function:
example_price = demand_based_model(
    base_price=10.0,
    occupancy=80,
    capacity=100,
    queue=5,
    traffic=0.7,
    special_day=1,
    vehicle_type='car'
)
print(f"Calculated price using demand_based_model: {example_price:.2f}")

Calculated price using demand_based_model: 13.65


In [ ]:
# Demonstrate the haversine function
lat1, lon1 = 40.7128, -74.0060  # New York City
lat2, lon2 = 34.0522, -118.2437 # Los Angeles
distance = haversine(lat1, lon1, lat2, lon2)
print(f"Distance between NYC and LA: {distance:.2f} km")

# Demonstrate the competitive_pricing function
# Sample data for current and nearby lots
current_lot_data = {
    'Latitude': 40.7580,
    'Longitude': -73.9855,
    'Occupancy': 70,
    'Capacity': 100,
    'Price': 15.0
}
current_lot = pd.Series(current_lot_data)

nearby_lots_data = {
    'Latitude': [40.7590, 40.7570, 40.7600],
    'Longitude': [-73.9860, -73.9845, -73.9870],
    'Occupancy': [80, 50, 95],
    'Capacity': [100, 80, 100],
    'Price': [14.0, 12.0, 16.0] # Include a 'Price' column
}
nearby_lots = pd.DataFrame(nearby_lots_data)

# Calculate competitive price
new_price = competitive_pricing(
    current_lot=current_lot,
    nearby_lots=nearby_lots,
    current_price=current_lot['Price'],
    occupancy=current_lot['Occupancy'],
    capacity=current_lot['Capacity']
)

print(f"Calculated competitive price: {new_price:.2f}")

Distance between NYC and LA: 3935.75 km
Calculated competitive price: 15.00


In [ ]:
base_price = 10
data['Price'] = base_price

for idx, row in data.iterrows():
    price = data.at[idx - 1, 'Price'] if idx > 0 else base_price

    # Model 1
    price = baseline_linear_model(price, row['Occupancy'], row['Capacity'])

    # Model 2
    price = demand_based_model(base_price, row['Occupancy'], row['Capacity'],
                               row['QueueLength'], row['TrafficLevel'],
                               row['IsSpecialDay'], row['VehicleType'])

    # Model 3 (Optional: Assuming nearby data available)
    nearby_lots = data[(data['TimeStep'] == row['TimeStep']) & (data.index != idx)]
    price = competitive_pricing(row, nearby_lots, price, row['Occupancy'], row['Capacity'])

    data.at[idx, 'Price'] = price


/tmp/ipython-input-29-4000668093.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '12.054246457171981' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[idx, 'Price'] = price


In [ ]:
display(data.head())

,ParkingLotID,TimeStep,Latitude,Longitude,Capacity,Occupancy,QueueLength,VehicleType,TrafficLevel,IsSpecialDay,Price
0,1,0,12.974908,77.575090,64,2,4,truck,0.719632,1,12.054246
1,1,1,13.090143,77.570635,69,68,1,truck,0.644363,0,10.784702
2,1,2,13.046399,77.665395,31,13,2,bike,0.033390,0,10.882499
3,1,3,13.019732,77.501912,32,7,9,truck,0.778205,1,15.000000
4,1,4,12.931204,77.659589,43,88,3,truck,0.207193,0,13.829543


In [ ]:
# Real-time Plotting Example
source = ColumnDataSource(data[data['ParkingLotID'] == 1][['TimeStep', 'Price']])

p = figure(title="Dynamic Pricing Over Time for Lot 1", x_axis_label='Time Step', y_axis_label='Price')
p.line(x='TimeStep', y='Price', source=source, line_width=2)

show(p)


In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource # Removed Histogram import

# Create a ColumnDataSource from the 'Price' column
price_source = ColumnDataSource(data[['Price']])

# Create a new plot figure
p = figure(title="Distribution of Parking Prices", x_axis_label='Price', y_axis_label='Frequency')

# Create a histogram glyph
hist, edges = np.histogram(price_source.data['Price'], bins=20) # You can adjust the number of bins
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")

# Show the plot
show(p)